In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(sigmoid())

    model.add(dense(label_dim, weight_init))

    return model

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\knh11\anaconda3\envs\tfpy3.10\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.1016, train_loss: 2.30172586, train_accuracy: 0.1328, test_Accuracy: 0.1135
Epoch: [ 0] [    1/  468] time: 1.3390, train_loss: 2.27851772, train_accuracy: 0.1016, test_Accuracy: 0.1135
Epoch: [ 0] [    2/  468] time: 1.6410, train_loss: 2.29397678, train_accuracy: 0.1328, test_Accuracy: 0.0974
Epoch: [ 0] [    3/  468] time: 1.9171, train_loss: 2.27235961, train_accuracy: 0.1016, test_Accuracy: 0.0980
Epoch: [ 0] [    4/  468] time: 2.1616, train_loss: 2.26800895, train_accuracy: 0.1094, test_Accuracy: 0.0980
Epoch: [ 0] [    5/  468] time: 2.4264, train_loss: 2.29149818, train_accuracy: 0.0859, test_Accuracy: 0.0981
Epoch: [ 0] [    6/  468] time: 2.6956, train_loss: 2.26302552, train_accuracy: 0.1953, test_Accuracy: 0.1851
Epoch: [ 0] [    7/  468] time: 2.9537, train_loss: 2.25819898, train_accuracy: 0.1641, test_Accuracy: 0.1778
Epoch: [ 0] [    8/  468] time: 3.1945, train_loss: 2.26842022, train_accuracy: 0.1562, test_Accuracy: 0.1864
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 20.6868, train_loss: 0.91099608, train_accuracy: 0.8047, test_Accuracy: 0.7791
Epoch: [ 0] [   76/  468] time: 20.9283, train_loss: 0.87465692, train_accuracy: 0.8203, test_Accuracy: 0.7772
Epoch: [ 0] [   77/  468] time: 21.2104, train_loss: 0.93570787, train_accuracy: 0.6953, test_Accuracy: 0.7845
Epoch: [ 0] [   78/  468] time: 21.4624, train_loss: 0.89623803, train_accuracy: 0.8125, test_Accuracy: 0.7973
Epoch: [ 0] [   79/  468] time: 21.7133, train_loss: 0.98948109, train_accuracy: 0.7188, test_Accuracy: 0.8097
Epoch: [ 0] [   80/  468] time: 22.0222, train_loss: 0.91261208, train_accuracy: 0.7969, test_Accuracy: 0.8172
Epoch: [ 0] [   81/  468] time: 22.2842, train_loss: 0.89673901, train_accuracy: 0.7656, test_Accuracy: 0.8208
Epoch: [ 0] [   82/  468] time: 22.4855, train_loss: 0.84768993, train_accuracy: 0.7891, test_Accuracy: 0.8249
Epoch: [ 0] [   83/  468] time: 22.7504, train_loss: 0.84294528, train_accuracy: 0.7969, test_Accuracy: 0.8231
E

Epoch: [ 0] [  149/  468] time: 39.5603, train_loss: 0.44323248, train_accuracy: 0.8984, test_Accuracy: 0.8842
Epoch: [ 0] [  150/  468] time: 39.8044, train_loss: 0.51622814, train_accuracy: 0.8672, test_Accuracy: 0.8840
Epoch: [ 0] [  151/  468] time: 40.0481, train_loss: 0.52690876, train_accuracy: 0.8438, test_Accuracy: 0.8849
Epoch: [ 0] [  152/  468] time: 40.2759, train_loss: 0.39568776, train_accuracy: 0.8906, test_Accuracy: 0.8850
Epoch: [ 0] [  153/  468] time: 40.5090, train_loss: 0.41970992, train_accuracy: 0.9062, test_Accuracy: 0.8857
Epoch: [ 0] [  154/  468] time: 40.7428, train_loss: 0.45793843, train_accuracy: 0.9062, test_Accuracy: 0.8869
Epoch: [ 0] [  155/  468] time: 41.0097, train_loss: 0.52146310, train_accuracy: 0.8359, test_Accuracy: 0.8873
Epoch: [ 0] [  156/  468] time: 41.2430, train_loss: 0.48402020, train_accuracy: 0.8750, test_Accuracy: 0.8869
Epoch: [ 0] [  157/  468] time: 41.5020, train_loss: 0.50796646, train_accuracy: 0.8594, test_Accuracy: 0.8869
E

Epoch: [ 0] [  223/  468] time: 58.2309, train_loss: 0.35415778, train_accuracy: 0.8984, test_Accuracy: 0.9013
Epoch: [ 0] [  224/  468] time: 58.5200, train_loss: 0.39994988, train_accuracy: 0.8750, test_Accuracy: 0.9048
Epoch: [ 0] [  225/  468] time: 58.7797, train_loss: 0.46134686, train_accuracy: 0.8750, test_Accuracy: 0.9035
Epoch: [ 0] [  226/  468] time: 59.0140, train_loss: 0.45434457, train_accuracy: 0.8281, test_Accuracy: 0.9030
Epoch: [ 0] [  227/  468] time: 59.2948, train_loss: 0.32159805, train_accuracy: 0.9219, test_Accuracy: 0.9029
Epoch: [ 0] [  228/  468] time: 59.5171, train_loss: 0.24545401, train_accuracy: 0.9297, test_Accuracy: 0.9013
Epoch: [ 0] [  229/  468] time: 59.7815, train_loss: 0.40365449, train_accuracy: 0.8984, test_Accuracy: 0.9009
Epoch: [ 0] [  230/  468] time: 60.0334, train_loss: 0.38065323, train_accuracy: 0.9062, test_Accuracy: 0.9014
Epoch: [ 0] [  231/  468] time: 60.2694, train_loss: 0.30657935, train_accuracy: 0.9375, test_Accuracy: 0.9040
E

Epoch: [ 0] [  297/  468] time: 76.8020, train_loss: 0.32753134, train_accuracy: 0.8984, test_Accuracy: 0.9120
Epoch: [ 0] [  298/  468] time: 77.0365, train_loss: 0.30837643, train_accuracy: 0.9453, test_Accuracy: 0.9129
Epoch: [ 0] [  299/  468] time: 77.2900, train_loss: 0.31056035, train_accuracy: 0.9141, test_Accuracy: 0.9136
Epoch: [ 0] [  300/  468] time: 77.5486, train_loss: 0.29154572, train_accuracy: 0.8906, test_Accuracy: 0.9137
Epoch: [ 0] [  301/  468] time: 77.7850, train_loss: 0.36743084, train_accuracy: 0.9141, test_Accuracy: 0.9131
Epoch: [ 0] [  302/  468] time: 78.0269, train_loss: 0.31584710, train_accuracy: 0.8984, test_Accuracy: 0.9134
Epoch: [ 0] [  303/  468] time: 78.2813, train_loss: 0.25927410, train_accuracy: 0.9219, test_Accuracy: 0.9129
Epoch: [ 0] [  304/  468] time: 78.5219, train_loss: 0.23604019, train_accuracy: 0.9219, test_Accuracy: 0.9125
Epoch: [ 0] [  305/  468] time: 78.7877, train_loss: 0.29701278, train_accuracy: 0.9062, test_Accuracy: 0.9120
E

Epoch: [ 0] [  371/  468] time: 95.5235, train_loss: 0.26822263, train_accuracy: 0.9141, test_Accuracy: 0.9126
Epoch: [ 0] [  372/  468] time: 95.7811, train_loss: 0.36098897, train_accuracy: 0.9141, test_Accuracy: 0.9136
Epoch: [ 0] [  373/  468] time: 96.0465, train_loss: 0.33839890, train_accuracy: 0.8984, test_Accuracy: 0.9154
Epoch: [ 0] [  374/  468] time: 96.2739, train_loss: 0.36600488, train_accuracy: 0.9062, test_Accuracy: 0.9172
Epoch: [ 0] [  375/  468] time: 96.5256, train_loss: 0.25993812, train_accuracy: 0.9219, test_Accuracy: 0.9181
Epoch: [ 0] [  376/  468] time: 96.7907, train_loss: 0.22906511, train_accuracy: 0.9141, test_Accuracy: 0.9186
Epoch: [ 0] [  377/  468] time: 97.0819, train_loss: 0.38370344, train_accuracy: 0.9141, test_Accuracy: 0.9183
Epoch: [ 0] [  378/  468] time: 97.3406, train_loss: 0.26522595, train_accuracy: 0.9297, test_Accuracy: 0.9187
Epoch: [ 0] [  379/  468] time: 97.6025, train_loss: 0.26369512, train_accuracy: 0.9297, test_Accuracy: 0.9181
E

Epoch: [ 0] [  445/  468] time: 114.2968, train_loss: 0.26304278, train_accuracy: 0.9219, test_Accuracy: 0.9234
Epoch: [ 0] [  446/  468] time: 114.5285, train_loss: 0.22044143, train_accuracy: 0.9375, test_Accuracy: 0.9228
Epoch: [ 0] [  447/  468] time: 114.7986, train_loss: 0.28551307, train_accuracy: 0.8906, test_Accuracy: 0.9224
Epoch: [ 0] [  448/  468] time: 115.0265, train_loss: 0.27369702, train_accuracy: 0.9453, test_Accuracy: 0.9236
Epoch: [ 0] [  449/  468] time: 115.2504, train_loss: 0.28412676, train_accuracy: 0.9062, test_Accuracy: 0.9240
Epoch: [ 0] [  450/  468] time: 115.5211, train_loss: 0.22483878, train_accuracy: 0.9297, test_Accuracy: 0.9250
Epoch: [ 0] [  451/  468] time: 115.7692, train_loss: 0.17950784, train_accuracy: 0.9609, test_Accuracy: 0.9252
Epoch: [ 0] [  452/  468] time: 116.0996, train_loss: 0.32145452, train_accuracy: 0.9219, test_Accuracy: 0.9248
Epoch: [ 0] [  453/  468] time: 116.3371, train_loss: 0.34053308, train_accuracy: 0.9062, test_Accuracy:

# Test accuracy : 92.36%